In [1]:
import torch
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import os
import sys
import random

root_path = os.path.dirname(os.getcwd())
print(root_path)
# sys.path.append(root_path)
# print(sys.path)

np.random.seed(101)

device = torch.device("cuda:1"if torch.cuda.is_available() else "cpu")
print(device)

def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True

setup_seed(20)

/home/chb/MachineLearning/ML-Project1
cuda:1


In [2]:

# Load dataset

X_train = torch.from_numpy(np.load("{}/data/X_train.npy".format(root_path)) / 255.)
X_test = torch.from_numpy(np.load("{}/data/X_test.npy".format(root_path)) / 255.)
y_train = np.load("{}/data/y_train.npy".format(root_path))
y_test = np.load("{}/data/y_test.npy".format(root_path))
y_train[y_train < 0] = 0
y_test[y_test < 0] = 0
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)
print(X_train.dtype, y_train.dtype)
X_train = torch.transpose(torch.transpose(X_train, 1, 3), 2, 3)
X_test = torch.transpose(torch.transpose(X_test, 1, 3), 2, 3)
print(X_train.shape, X_test.shape)
print(X_train.dtype)

train_set = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_set, batch_size=8, shuffle=True, num_workers=2)
test_set = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_set, batch_size=8, shuffle=False, num_workers=2)


torch.Size([20680, 96, 96, 3]) torch.Size([5203, 96, 96, 3])
torch.Size([20680]) torch.Size([5203])
torch.float64 torch.float64
torch.Size([20680, 3, 96, 96]) torch.Size([5203, 3, 96, 96])
torch.float64


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 7)
        self.conv3 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32*8*8, 600)
        self.fc2 = nn.Linear(600, 150)
        self.fc3 = nn.Linear(150, 18)
        self.fc4 = nn.Linear(18, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 32*8*8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    
net = Net()
net.to(device)


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(7, 7), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2048, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=18, bias=True)
  (fc4): Linear(in_features=18, out_features=2, bias=True)
)

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
for epoch in range(7):
    
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
#         print(inputs.shape)
        optimizer.zero_grad()
        
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i%200 == 199:
            print("[{}, {}] loss: {:.5f}".format(epoch+1, i+1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1, 200] loss: 0.58238
[1, 400] loss: 0.51291
[1, 600] loss: 0.51562
[1, 800] loss: 0.48421
[1, 1000] loss: 0.51170
[1, 1200] loss: 0.48864
[1, 1400] loss: 0.47753
[1, 1600] loss: 0.50397
[1, 1800] loss: 0.53568
[1, 2000] loss: 0.49087
[1, 2200] loss: 0.49462
[1, 2400] loss: 0.49063
[2, 200] loss: 0.49151
[2, 400] loss: 0.46348
[2, 600] loss: 0.47487
[2, 800] loss: 0.50895
[2, 1000] loss: 0.44918
[2, 1200] loss: 0.41098
[2, 1400] loss: 0.35033
[2, 1600] loss: 0.25956
[2, 1800] loss: 0.21359
[2, 2000] loss: 0.19365
[2, 2200] loss: 0.18501
[2, 2400] loss: 0.17770
[3, 200] loss: 0.11125
[3, 400] loss: 0.12095
[3, 600] loss: 0.13034
[3, 800] loss: 0.10758
[3, 1000] loss: 0.11093
[3, 1200] loss: 0.11770
[3, 1400] loss: 0.11877
[3, 1600] loss: 0.08201
[3, 1800] loss: 0.09349
[3, 2000] loss: 0.08397
[3, 2200] loss: 0.09326
[3, 2400] loss: 0.05889
[4, 200] loss: 0.07888
[4, 400] loss: 0.07704
[4, 600] loss: 0.06855
[4, 800] loss: 0.08268
[4, 1000] loss: 0.07209
[4, 1200] loss: 0.07248
[4, 1400

In [8]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
#         print(labels)
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.long()).sum().item()

print("Acc on {} test samples is: {:.2f}%".format(total, 100*correct/total))

# torch.save(net, '{}/data/mycnn.pkl'.format(root_path))
torch.save(net.state_dict(), '{}/data/cnn_model.pkl'.format(root_path))

Acc on 5203 test samples is: 98.21%
